In [2]:
import os
import os.path as osp
import numpy as np
import cv2
from glob import glob
import mylib.polSAR_utils as psr
import mylib.image_utils as iu
from mylib.types import LINE_SEPARATOR
import mylib.labelme_utils as lu
import json
import pickle
import re

import solaris as sol
import solaris.preproc.pipesegment as pipesegment
import solaris.preproc.label as label
import solaris.preproc.image as image
import solaris.preproc.sar as sar
from solaris.data import data_dir
import skimage
import geopandas as gpd
from matplotlib import pyplot as plt
from shapely.ops import cascaded_union

[INFO   ] env:<module>:57 - failed to import boto3, continuing.
/home/csl/anaconda3/envs/solaris/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/csl/anaconda3/envs/solaris/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/csl/anaconda3/envs/solaris/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.

In [4]:
a=np.array([[1,2],[3,2]])
a, a.size

(array([[1, 2],
        [3, 2]]),
 4)

In [2]:
path = r'/home/csl/code/preprocess/data/SN6_extend'
slc_path = osp.join(path, r'newSLC2')
label_path = osp.join(path, r'geojson_buildings')
tmp_dir = r'/home/csl/code/preprocess/tmp'
rgb_file_path = r'/home/csl/code/preprocess/data/SN6_extend/PS-RGB/SN6_AOI_11_Rotterdam_PS-RGB_TrainVal.tif'
label_files_path = [r'/home/csl/code/preprocess/data/SN6_extend/geojson_buildings/SN6_AOI_11_Rotterdam_Buildings_GT20sqm-TrainVal.geojson', r'/home/csl/code/preprocess/data/SN6_extend/geojson_buildings/SN6_AOI_11_Rotterdam_Buildings_GT20sqm-Test.geojson']
label_file_path = r'/home/csl/code/preprocess/data/SN6_extend/geojson_buildings/SN6_AOI_11_Rotterdam_Buildings_GT20sqm-TrainValTest.geojson'

In [3]:
tile_size = 900
tile_slc_path = osp.join(path, r'tile_slc', str(tile_size))
tile_rgb_path = osp.join(path, r'tile_rgb', str(tile_size))
tile_label_path = osp.join(path, r'tile_label', str(tile_size))
tile_mask_path = osp.join(path, r'tile_mask', str(tile_size))
os.makedirs(tile_slc_path, exist_ok=True)
os.makedirs(tile_rgb_path, exist_ok=True)
os.makedirs(tile_label_path, exist_ok=True)
os.makedirs(tile_mask_path, exist_ok=True)

In [4]:
# tile PS-RGB
rgb_tiler = sol.tile.raster_tile.RasterTiler(
            dest_dir=tile_rgb_path,
            src_tile_size=(tile_size, tile_size),
            verbose=True)
rgb_bounds_crs = rgb_tiler.tile(rgb_file_path)

Initializing Tiler...
Tiler initialized.
dest_dir: /home/csl/code/preprocess/data/SN6_extend/tile_rgb/900
dest_crs will be inferred from source data.
src_tile_size: (900, 900)
tile size units metric: False
Resampling is set to None
Beginning tiling...


0it [00:00, ?it/s]

Checking input data...
[1, 2, 3]
Source CRS: EPSG:32631
Destination CRS: EPSG:32631
Inputs OK.
reading data from window
None
reading data from window
None
reading data from window
None
reading data from window
None
reading data from window
None
reading data from window
None
reading data from window
None
reading data from window
None
reading data from window
None
reading data from window
None
reading data from window
None
reading data from window
None
reading data from window
None
reading data from window
None
reading data from window
None
reading data from window
None
reading data from window
None
reading data from window
None
reading data from window
None
reading data from window
None
reading data from window
None
reading data from window
None
reading data from window
None
reading data from window
None
reading data from window
None
reading data from window
None
reading data from window
None
reading data from window
None
reading data from window
None
reading data from window
None
readi

In [5]:
# tile label
vector_tiler = sol.tile.vector_tile.VectorTiler(
                dest_dir=tile_label_path,
                verbose=True)
vector_tiler.tile(label_file_path,
            tile_bounds=rgb_tiler.tile_bounds,
            tile_bounds_crs=rgb_bounds_crs)

Preparing the tiler...
Initialization done.


0it [00:00, ?it/s]

Num tiles: 494


[ERROR  ] collection:__init__:157 - %s
[ERROR  ] collection:__init__:157 - %s
[WARNING] collection:__init__:157 - %s
[WARNING] collection:__init__:157 - %s
[ERROR  ] geos:callback:261 - %s
[ERROR  ] geos:callback:261 - %s
[ERROR  ] geos:callback:261 - %s
[ERROR  ] geos:callback:261 - %s
[ERROR  ] geos:callback:261 - %s
[ERROR  ] geos:callback:261 - %s
[ERROR  ] geos:callback:261 - %s
[ERROR  ] geos:callback:261 - %s
[ERROR  ] geos:callback:261 - %s
[ERROR  ] geos:callback:261 - %s
[ERROR  ] geos:callback:261 - %s
[ERROR  ] geos:callback:261 - %s
[ERROR  ] geos:callback:261 - %s
[ERROR  ] geos:callback:261 - %s
[ERROR  ] geos:callback:261 - %s
[ERROR  ] geos:callback:261 - %s
[ERROR  ] geos:callback:261 - %s
[ERROR  ] geos:callback:261 - %s
[ERROR  ] geos:callback:261 - %s
[ERROR  ] geos:callback:261 - %s
[ERROR  ] geos:callback:261 - %s
[ERROR  ] geos:callback:261 - %s
[ERROR  ] geos:callback:261 - %s
[ERROR  ] geos:callback:261 - %s
[ERROR  ] geos:callback:261 - %s
[ERROR  ] geos:call

In [ ]:
tile_label_path

'/home/csl/code/preprocess/data/SN6_extend/tile_label/900'

In [ ]:
# change geojson to mask
labels = glob(osp.join(tile_label_path, r'*.geojson'))
label_prefix = r'geoms_'
label_suffix = r'geojson'
rgb_prefix = r'SN6_AOI_11_Rotterdam_PS-RGB_TrainVal_'
rgb_suffix = r'tif'
PALETTE = np.array([[0, 0, 0], [255, 255, 255]])
for label in labels:
    rgb = label.replace(label_prefix, rgb_prefix).replace(label_suffix, rgb_suffix).replace(tile_label_path, tile_rgb_path)
    mask = sol.vector.mask.footprint_mask(df=label, reference_im=rgb)
    mask = mask > 0
    mask_path = osp.join(tile_mask_path, osp.basename(label).replace(label_suffix, 'png'))
    # print(f'saving {mask_path}')
    lu.lblsave(mask_path, mask, colormap=PALETTE)


Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point 

saving /home/csl/code/preprocess/data/SN6_extend/tile_mask/900/geoms_590461_5752208.png
saving /home/csl/code/preprocess/data/SN6_extend/tile_mask/900/geoms_595411_5754008.png
saving /home/csl/code/preprocess/data/SN6_extend/tile_mask/900/geoms_590011_5747258.png
saving /home/csl/code/preprocess/data/SN6_extend/tile_mask/900/geoms_597211_5749508.png


Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point 

saving /home/csl/code/preprocess/data/SN6_extend/tile_mask/900/geoms_595411_5752658.png


Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point 

...

Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point 

saving /home/csl/code/preprocess/data/SN6_extend/tile_mask/900/geoms_590911_5749058.png
saving /home/csl/code/preprocess/data/SN6_extend/tile_mask/900/geoms_594061_5752658.png
saving /home/csl/code/preprocess/data/SN6_extend/tile_mask/900/geoms_597211_5747258.png


Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point 

saving /home/csl/code/preprocess/data/SN6_extend/tile_mask/900/geoms_596311_5750408.png
saving /home/csl/code/preprocess/data/SN6_extend/tile_mask/900/geoms_600361_5746358.png


Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point or LineString
Geometry must be a Point 

In [ ]:
label_path = r'/home/csl/code/preprocess/data/SN6_extend/tile_label/900/geoms_590011_5746358.geojson'
osp.basename(label_path)

'geoms_590011_5746358.geojson'

In [ ]:
slcs = glob(osp.join(slc_path, r'slc_*.tif'))
print(f'len{slcs} samples in total')
for slc in slcs:
    timestamp = re.findall(r'\d{14}\_\d{14}')
    assert len(timestamp)==0, f'find {len(timestamp)} matches'
    timestamp = timestamp[0]
    slc_tiler = sol.tile.raster_tile.RasterTiler(
        dest_dir=tile_slc_path,
        verbose=True,
        src_tile_size=(tile_size, tile_size),
        resampling='bilinear',
        tile_bounds=rgb_tiler.tile_bounds,
    )
    slc_tiler.tile(slc)
    break